# Notebook para tratamento dos dados

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#style.use('seaborn')
%matplotlib inline


In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from sklearn import preprocessing, metrics
# !pip install -U scikit-fuzzy
import sys
import skfuzzy as fuzz
sys.setrecursionlimit(1000)

In [3]:
data_column_names = ["Date", "Time", "Epoch", "Moteid", "Temperature", "Humidity", "Light", "Voltage"]
dplot = pd.read_csv("../data.txt", sep = ' ', names = data_column_names)

S12df = dplot[dplot['Moteid']==12]
S13df = dplot[dplot['Moteid']==13]

In [5]:
S11df = pd.read_csv('dfImputation.csv')
S11df.shape,S12df.shape,S13df.shape

((65581, 8), (23078, 8), (31505, 8))

In [6]:
S11df['3'] = 11.0
S11df.head()

,0,1,2,3,4,5,6,7
0,2004-03-21,19:06:38.75603,0,11.0,21.772,49.25850,478.40,2.32000
1,2004-03-31,03:38:15.757551,1,11.0,0.000,-3.91901,11.04,2.03397
2,2004-02-28,00:59:16.02785,2,11.0,0.000,37.09330,45.08,2.69964
3,2004-02-28,01:03:16.33393,3,11.0,0.000,38.46290,45.08,2.68742
4,2004-02-28,01:06:16.013453,4,11.0,0.000,38.80390,45.08,2.68742


In [7]:
outliersIL = []
outliersILValues = []
for i, row in S12df.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL.append(i)
        outliersILValues.append(row['Temperature'])
print(len(outliersIL))


3467


In [8]:
# Removendo outliers no Sensor
passo3= S12df.copy()
print(passo3.shape)
passo3= passo3.drop(outliersIL)
print(passo3.shape)
passo3.to_csv('S12FINAL.csv')

(23078, 8)
(19611, 8)


In [9]:
outliersIL2 = []
outliersILV2 = []
for i, row in S13df.iterrows():
    if row['Temperature'] >=26.2:
        outliersIL2.append(i)
        outliersILV2.append(row['Temperature'])
print(len(outliersIL2))


6390


In [10]:
# Removendo outliers no S2
passo3_3= S13df.copy()
print(passo3_3.shape)
passo3_3= passo3_3.drop(outliersIL2)
print(passo3_3.shape)
passo3_3.to_csv('S13FINAL.csv')

(31505, 8)
(25115, 8)


In [11]:
outliersIL_3 = []
outliersILV_3 = []
for i, row in S11df.iterrows():
    if row['4'] >=26.2:
        outliersIL_3.append(i)
        outliersILV_3.append(row['4'])
print(len(outliersIL_3))


0


In [12]:
# Removendo outliers no sensor
passo3_2= S11df.copy()
print(passo3_2.shape)
passo3_2= passo3_2.drop(outliersIL_3)
print(passo3_2.shape)
passo3_2.to_csv('S11FINAL.csv')

(65581, 8)
(65581, 8)


In [13]:
S11df_ = pd.read_csv('S11FINAL.csv')
S12df_ = pd.read_csv('S12FINAL.csv')
S13df_ = pd.read_csv('S13FINAL.csv')
S11df_['3'] = 11
S11df_.shape, S12df_.shape, S13df_.shape

((65581, 9), (19611, 9), (25115, 9))

In [14]:
S11df_.head()

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,2004-03-21,19:06:38.75603,0,11,21.772,49.25850,478.40,2.32000
1,1,2004-03-31,03:38:15.757551,1,11,0.000,-3.91901,11.04,2.03397
2,2,2004-02-28,00:59:16.02785,2,11,0.000,37.09330,45.08,2.69964
3,3,2004-02-28,01:03:16.33393,3,11,0.000,38.46290,45.08,2.68742
4,4,2004-02-28,01:06:16.013453,4,11,0.000,38.80390,45.08,2.68742


# Sistema Fuzzy

In [15]:
# Modelo Fuzzy
temp = np.arange(-26.2, 26.2, 0.1)
tempF = np.arange(-26.2, 26.2, 0.1)

# Generate fuzzy membership functions
temp_lo = fuzz.trimf(temp, [-26.2, 0.0, 5.0])  
temp_md = fuzz.trimf(temp, [0.0, 5.0, 20.0])      
temp_hi = fuzz.trimf(temp, [17.20, 22.8, 26.2]) 


mfx = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2])


tempFinal_lo = fuzz.trapmf(tempF, [-26.2, 5.0, 17.20, 26.2]) 
tempFinal_md = fuzz.trimf(tempF, [0, 5.0, 20.0]) 
tempFinal_hi = fuzz.trimf(tempF, [17.20, 22.8, 26.2])


In [17]:
# # Visualize these universes and membership functions
# fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(8, 9))

# ax0.plot(temp, temp_lo, 'b', linewidth=1.5, label='Baixa')
# ax0.plot(temp, temp_md, 'g', linewidth=1.5, label='Média')
# ax0.plot(temp, temp_hi, 'r', linewidth=1.5, label='Alta')
# ax0.set_title('Temperature')
# ax0.legend()

# ax1.plot(tempF, tempFinal_lo, 'b', linewidth=1.5, label='Baixa')
# ax1.plot(tempF, tempFinal_md, 'g', linewidth=1.5, label='Média')
# ax1.plot(tempF, tempFinal_hi, 'r', linewidth=1.5, label='Alta')
# ax1.set_title('Final - Temperature ')
# ax1.legend()

# # Turn off top/right axes
# for ax in (ax0, ax1):
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.get_xaxis().tick_bottom()
#     ax.get_yaxis().tick_left()

# plt.tight_layout()

In [18]:
def fuzification(i, data, data1, data2): 
    if data['3'].ix[i]== 11:
        temp_level_lo0 = fuzz.interp_membership(temp, temp_lo,  data1['Temperature'][i]) # ex: [52, [51, 52]]
        temp_level_md0 = fuzz.interp_membership(temp, temp_md,  data1['Temperature'][i])
        temp_level_hi0 = fuzz.interp_membership(temp, temp_hi,  data1['Temperature'][i])

        temp_level_lo1 = fuzz.interp_membership(temp, temp_lo,  data2['Temperature'][i])
        temp_level_md1 = fuzz.interp_membership(temp, temp_md,  data2['Temperature'][i])
        temp_level_hi1 = fuzz.interp_membership(temp, temp_hi,  data2['Temperature'][i])

        active_rule1_0 = np.fmin(temp_level_lo0, temp_level_lo1)
        active_rule1_1 = np.fmin(temp_level_lo0, temp_level_md1)
        active_rule1_2 = np.fmin(temp_level_lo0, temp_level_hi1)
        active_rule1_3 = np.fmin(temp_level_md0, temp_level_lo1)
        active_rule1_4 = np.fmin(temp_level_md0, temp_level_md1)
        active_rule1_5 = np.fmin(temp_level_md0, temp_level_hi1)
        active_rule1_6 = np.fmin(temp_level_hi0, temp_level_lo1)
        active_rule1_7 = np.fmin(temp_level_hi0, temp_level_md1)
        active_rule1_8 = np.fmin(temp_level_hi0, temp_level_hi1)


        active_rule1_Final_0 = np.fmax(active_rule1_0, tempFinal_lo)
        active_rule1_Final_1 = np.fmax(active_rule1_1, tempFinal_lo)
        active_rule1_Final_2 = np.fmax(active_rule1_2, tempFinal_md)
        active_rule1_Final_3 = np.fmax(active_rule1_3, tempFinal_lo)
        active_rule1_Final_4 = np.fmax(active_rule1_4, tempFinal_hi)
        active_rule1_Final_5 = np.fmax(active_rule1_5, tempFinal_hi)
        active_rule1_Final_6 = np.fmax(active_rule1_6, tempFinal_md)
        active_rule1_Final_7 = np.fmax(active_rule1_7, tempFinal_hi)
        active_rule1_Final_8 = np.fmax(active_rule1_8, tempFinal_hi)

        r1_0 = np.fmax(active_rule1_Final_8, np.fmax(active_rule1_Final_7, np.fmax(active_rule1_Final_6, active_rule1_Final_5) ) )  
        r1_1 = np.fmax(active_rule1_Final_5, np.fmax(active_rule1_Final_3, np.fmax(active_rule1_Final_2, np.fmax(active_rule1_Final_1, active_rule1_Final_0)) ) )  
        aggregated = np.fmax(r1_0,r1_1)
        return aggregated  


In [19]:
def defuzz(x, mfx, mode):
    """
    Defuzzification of a membership function, returning a defuzzified value
    of the function at x, using various defuzzification methods.

    Parameters
    ----------
    x : 1d array or iterable, length N
        Independent variable.
    mfx : 1d array of iterable, length N
        Fuzzy membership function.
    mode : string
        Controls which defuzzification method will be used.
        * 'centroid': Centroid of area
        * 'bisector': bisector of area
        * 'mom'        : mean of maximum
        * 'som'        : min of maximum
        * 'lom'        : max of maximum

    Returns
    -------
    u : float or int
        Defuzzified result.

    See also
    --------
    CENTROID, DCENTROID

    """
    mode = mode.lower()
    x = x.ravel()
    mfx = mfx.ravel()
    n = len(x)
    assert n == len(mfx), 'Length of x and fuzzy membership function must be \
                          identical.'

    if 'Centroid' in mode or 'bisector' in mode:
        tot_area = mfx.sum()
        assert tot_area != 0, 'Total area is zero in defuzzification!'

        if 'Centroid' in mode:
            return centroid(x, mfx)

        elif 'bisector' in mode:
            tmp = 0
            for k in range(n):
                tmp += mfx[k]
                if tmp >= tot_area / 2.:
                    return x[k]

    elif 'mom' in mode:
        return np.mean(x[mfx == mfx.max()])

    elif 'som' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).min()][0]

    elif 'lom' in mode:
        tmp = x[mfx == mfx.max()]
        return tmp[tmp == np.abs(tmp).max()][0]

    else:
        raise ValueError('The input for `mode`, %s, was incorrect.' % (mode))



In [20]:
aux = 0.0
for i, row in S11df_.iterrows():
        if S11df_['4'][i] == 0.0:            
            try:
                S11df_.at[i, '4'] = defuzz(tempF,fuzification(i, S11df_, S12df_, S13df_ ), 'lom')
                aux = defuzz(tempF,fuzification(i, S11df_, S12df_, S13df_ ), 'lom')
            except:
                S11df_.at[i, '4'] = aux
                
S11df_.tail()

,Unnamed: 0,0,1,2,3,4,5,6,7
65576,65576,2004-03-27,12:25:21.115094,65531,11,17.100,-3.91901,596.16,2.19159
65577,65577,2004-03-27,12:25:50.23894,65532,11,17.100,-3.91901,596.16,2.19159
65578,65578,2004-03-27,12:26:13.504686,65533,11,17.100,-3.91901,596.16,2.19159
65579,65579,2004-03-14,20:28:35.869938,65534,11,17.100,42.48440,41.40,2.51661
65580,65580,2004-03-21,19:05:55.228199,65535,11,21.772,49.22620,478.40,2.32000


In [21]:
print(S11df_.shape)
S11df_.to_csv('finalsensor.csv', index=False)


(65581, 9)


# Final